# Treinamento do modelo

Dataset: https://www.kaggle.com/datasets/dennisho/blackjack-hands/

Modelo: MLP (sem o uso de card counting)

## 1) Tratamento dos dados

In [3]:
import pandas as pd

rows = 50000
df = pd.read_csv('blackjack_simulator.csv', sep=',', nrows=rows)

# Remove as colunas não utilizadas
df = df.drop(columns=['shoe_id', 'run_count', 'true_count', 'cards_remaining', 'dealer_up', 'initial_hand', 'dealer_final_value', 'player_final_value'])

# Filtra os dados por ganho
df = df[df['win'] >= 1]

# Transforma as strings em listas
df['dealer_final'] = df['dealer_final'].apply(lambda x: eval(x)[0])  # Só a primeira carta do dealer será utilizada para emular as informações que um jogador tem
df['player_final'] = df['player_final'].apply(lambda x: eval(x))
df['actions_taken'] = df['actions_taken'].apply(lambda x: eval(x))
df

,dealer_final,player_final,actions_taken,win
0,10,"[[10, 11]]",[[S]],1.5
1,10,"[[5, 5, 11]]","[[H, S]]",1.0
2,6,"[[3, 10]]",[[S]],1.0
6,6,"[[3, 2, 10], [3, 4, 5]]","[[P, H, S], [H, S]]",2.0
8,5,"[[8, 10]]",[[S]],1.0
...,...,...,...,...
49984,11,"[[5, 5, 10]]","[[N, H, S]]",1.0
49988,3,"[[3, 10]]",[[S]],1.0
49990,8,"[[4, 6, 9]]",[[D]],2.0
49991,10,"[[7, 6, 6]]","[[H, S]]",1.0


In [4]:
# Divide as ações dos players em vários estados de jogo
for i in range(0, len(df.index)):
    linha = df.loc[i]

    for j in range(0, len(linha['actions_taken'])):
        mao = linha['player_final'][j]
        acoes = linha['actions_taken'][j]

        # print({'dealer_final': linha['dealer_final'], 'player_final': mao[:4], 'actions_taken': acoes[0]})

        for k in range(0, len(acoes)):
            acao = acoes[k]

            df = pd.concat([df, pd.DataFrame([{'dealer_final': linha['dealer_final'], 'player_final': sum(mao[:2+k]), 'actions_taken': acoes[k], 'win': linha['win']}])], ignore_index=True)

df = df.drop(range(0, rows))
df


,dealer_final,player_final,actions_taken,win
50000,10,21,S,1.0
50001,6,10,D,2.0
50002,10,19,S,1.0
50003,2,18,S,1.0
50004,3,16,S,1.0
...,...,...,...,...
52202,3,13,S,1.0
52203,8,10,D,2.0
52204,10,13,H,1.0
52205,10,19,S,1.0


## 2) Treinamento

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X = df.drop(['actions_taken', 'win'], axis=1)
y = df['actions_taken']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

clf = MLPClassifier().fit(X_train, y_train)
clf.score(X_test, y_test)

/home/erick/Documentos/Projeto_BlackJack/venv/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.8043478260869565

In [13]:
type(X_train[:1])

pandas.core.frame.DataFrame

In [12]:
clf.predict(X_train[:1])[0]

np.str_('S')

## 3) Exportação

In [8]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(clf, f)